In [1]:
import Constants as Const
import pandas as pd
import numpy as np
import DataLoader

ISSUE_ID = 'user_feedback_id'
MATCHING = 'Matching Bug Report'
SIMILARITY = 'Similarity'
FOUND_BUG = 'found_relevant_issue_manually'


df = DataLoader.load_resolved_sheet()
df = df[~df[FOUND_BUG]]

In [5]:
from sklearn.metrics import average_precision_score

def calculate_average_precision(issue_id, n=3):
    scores = df[df[ISSUE_ID]==issue_id][[MATCHING, SIMILARITY]][:n]
    ap = average_precision_score(scores[MATCHING], scores[SIMILARITY])
    return ap if not np.isnan(ap) else 0

def calculate_overall_at(n=3):
    total = 0
    issue_ids = df[ISSUE_ID].unique()
    for issue_id in issue_ids:
        ap = calculate_average_precision(issue_id, n)
        total += ap

    mean_average_precision = total/len(issue_ids)
    print(f'{f"Overall Mean Average Precision@{n}:":<35} {mean_average_precision:.2f}')
    print()

calculate_overall_at(3)
calculate_overall_at(2)
calculate_overall_at(1)

def calculate_map_at(n):
    for app in Const.APPS:
        app_matches = df[df['App']==app]
        issue_ids = app_matches[ISSUE_ID].unique()
        total = 0
        for issue_id in issue_ids:
            ap = calculate_average_precision(issue_id, n)
            total += ap
        app_mean_average_precision = total/len(issue_ids)
        print(f'{app+f" Mean Average Precision@{n}:":<35} {app_mean_average_precision:>4.2f}')

calculate_map_at(3)
print()
calculate_map_at(2)
print()
calculate_map_at(1)



Overall Mean Average Precision@3:   0.55

Overall Mean Average Precision@2:   0.53

Overall Mean Average Precision@1:   0.45

Firefox Mean Average Precision@3:   0.58
VLC Mean Average Precision@3:       0.40
Signal Mean Average Precision@3:    0.50
Nextcloud Mean Average Precision@3: 0.73

Firefox Mean Average Precision@2:   0.54
VLC Mean Average Precision@2:       0.38
Signal Mean Average Precision@2:    0.47
Nextcloud Mean Average Precision@2: 0.73

Firefox Mean Average Precision@1:   0.50
VLC Mean Average Precision@1:       0.32
Signal Mean Average Precision@1:    0.38
Nextcloud Mean Average Precision@1: 0.62


In [3]:
length=len(df)
df['rank']=-1

for i in range(length):
    df.iat[i, list(df.columns).index('rank')]=int(i%3)+1

def calculate_hit_ratio_at(df, n=3):
    groups = df.groupby(['App', 'user_feedback_id']).agg({
        MATCHING: np.count_nonzero
    })
    groups.reset_index(inplace=True)
    values = pd.pivot_table(groups, index=['App'], columns=MATCHING, aggfunc='size', fill_value=0)

    hits = values[values.columns[1:]].sum(axis=1) if len(values.columns)>2 else values[1]
    total = values[0]+hits
    hit_ratio = hits/total

    print(f'Hit Ratio@{n}')
    print(hit_ratio.round(2))

rank3 = df
rank2 = df[df['rank']<=2]
rank1 = df[df['rank']<=1]
calculate_hit_ratio_at(rank3, 3)
calculate_hit_ratio_at(rank2, 2)
calculate_hit_ratio_at(rank1, 1)

Hit Ratio@3
App
Firefox      0.74
Nextcloud    0.89
Signal       0.68
VLC          0.51
dtype: float64
Hit Ratio@2
App
Firefox      0.58
Nextcloud    0.84
Signal       0.57
VLC          0.44
dtype: float64
Hit Ratio@1
App
Firefox      0.50
Nextcloud    0.62
Signal       0.38
VLC          0.32
dtype: float64
